In [1]:
import sys
print(sys.prefix)

/Volumes/Mac/opt/anaconda3/envs/nova


In [2]:
import numpy as np
print(np.__version__)

IOStream.flush timed out


1.26.4


In [3]:
import re
import textwrap

In [4]:
with open('lunch.txt', 'r') as f:
    lunch = f.read() 
    
with open('nova.txt', 'r') as f:
    nova = f.read()

with open('ticket.txt', 'r') as f:
    ticket = f.read() 

with open('soft.txt', 'r') as f:
    soft = f.read() 

with open('zone.txt', 'r') as f:
    zone = f.read() 

In [5]:
soft2 = set(soft.lower().split())
ticket2 = set(ticket.lower().split())
nova2 = set(nova.lower().split())
lunch2 = set(lunch.lower().split())
zone2 = set(zone.lower().split())

intersection1 = lunch2 & soft2
intersection2 = lunch2 & nova2
intersection3 = lunch2 & zone2
intersection4 = lunch2 & ticket2
intersection5 = soft2 & nova2
intersection6 = soft2 & zone2
intersection7 = soft2 & ticket2
intersection8 = nova2 & ticket2
intersection9 = nova2 & zone2
intersection10 = zone2 & ticket2

cutups1 = intersection1.union(intersection2, intersection3, intersection4, intersection5)
cutups2 = intersection6.union(intersection7, intersection8, intersection9, intersection10)
hoard = cutups1.union(cutups2)
print(hoard)

{'larcenous', 'corsica', 'pearls', 'know,"', 'tear', 'deserves', 'splash', 'fixers', 'constructed', "i'd", 'hyphenated', '73', 'chap', 'palm', 'heart', 'moves', '37', 'formed', 'substance', '"only', 'happening', 'iridescent', 'ask', 'spinning', 'underneath', 'way.', 'kid,"', 'butterfly', 'driving', 'alcohol.', 'ass', 'floats', 'data', 'pantless', 'dirt', 'no-smell', 'reaches', 'thereby', 'negative', 'hydrochloric', '110', 'script', 'infinite', 'bleeding', '36', 'l', 'oven', 'ate', 'town.', 'bait', 'slug', 'elevated', 'flies.', 'terminal,', 'races', 'swing', 'become', 'master,', 'tube.', 'injections', 'wards', 'also', 'writes', 'for,', 'story', 'begging', "lookin'", 'molten', 'round', 'forming', 'lovemaking', 'religious', '32', 'idiosyncracies,', 'corners', 'dallas', 'roger', 'canals,', 'imported', 'organism', 'freelandt', 'not', 'needs', 'tried', 'pissed', 'twisted', 'vases', 'paper-thin', 'long.', 'repellent,', 'all,', 'doling', 'lying', 'picture', 'battered', 'matter.', 'fellow', 'sl

In [6]:
hoard = list(hoard)
hoard = [x for x in hoard if not (x.isdigit() 
        or x[0] == '-' and x[1:].isdigit())]
chars_to_remove = [".",",","?","!",":","(",")"]
hoard = ["".join(ch for ch in string if ch not in chars_to_remove) for string in hoard]
hoard = [x for x in hoard if len(x) > 1]
hoard = [item.strip('"') for item in hoard]
hoard = [s for s in hoard if not s.startswith("-")]
hoard = [s for s in hoard if not s.endswith("-")]

In [7]:
with open(r'hoard.txt', 'w') as fp:
    for item in hoard:
        fp.write("%s\n" % item)
    print('Done')

Done


In [8]:
with open("hoard.txt", "r") as f:
    word_list = [line.strip() for line in f if line.strip()]

In [9]:
naked_lunch = lunch.lower()
with open(r'naked_lunch.txt', 'w') as fp:
    fp.write(naked_lunch)
print('Done')

Done


In [10]:
print(naked_lunch)

 i awoke from the sickness at the age of forty-five, calm and sane, and in reasonably good health except for a weakened liver and the look of borrowed flesh common to all who survive. the sickness... most survivors do not remember the delirium in detail. i apparently took detailed notes on sickness and delirium. i have no precise memory of writing the notes which have now been published under the title naked lunch. the title was suggested by jack kerouac. i did not understand what the title meant until my recent recovery. the title means exactly what the words say: naked lunch -- a frozen moment when everyone sees what is on the end of every fork. the sickness is drug addiction and i was an addict for fifteen years. when i say addict i mean an addict to junk (generic term for opium and/or derivatives including all synthetics from demerol to palfium). i have used junk in many forms: morphin, heroin, dilaudid, eukodol, pantopon, diocodid, diosane, opium, demerol, dolophine, palfium. i ha

In [11]:
## Model training time

In [26]:
import torch

In [27]:
from torch.utils.data import Dataset

In [40]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, LogitsProcessorList, LogitsProcessor

In [59]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

In [119]:
##
## TOKENIZING AND TRAINING NAKED LUNCH
##

In [151]:
with open('lunch.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    
tokenizer.pad_token = tokenizer.eos_token

input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, padding=True)

In [152]:
text = text.replace('\n', ' ') 

input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, padding=True)

print(f"Tokenized text length: {len(input_ids[0])}")

Tokenized text length: 1024


In [153]:
class TextDataset(Dataset):
    def __init__(self, input_ids, block_size):
        self.input_ids = input_ids[0]
        self.block_size = block_size
        self.num_samples = len(self.input_ids) // self.block_size  
        
        print(f"Total tokens: {len(self.input_ids)}, Number of samples: {self.num_samples}")

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        start_idx = idx * self.block_size
        end_idx = start_idx + self.block_size
        input_ids_block = self.input_ids[start_idx:end_idx]
        return input_ids_block, input_ids_block

block_size = 512 
dataset = TextDataset(input_ids, block_size)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch in dataloader:
    print(batch)

Total tokens: 1024, Number of samples: 2
[tensor([[  611,   345,  5457,  ..., 34593,  1202,    13],
        [  314, 43363,   422,  ...,  1011,  2279,   736]]), tensor([[  611,   345,  5457,  ..., 34593,  1202,    13],
        [  314, 43363,   422,  ...,  1011,  2279,   736]])]


In [ ]:
from transformers import GPT2LMHeadModel
from torch.optim import AdamW
from torch.nn import functional as F

# Loading gpt2 with the appropriate pad_token_id
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

# Setting model to train
model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
epochs = 32
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for batch_idx, (input_ids, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        # Move tensors to device (e.g., CUDA if available)
        input_ids = input_ids.to(model.device)
        labels = labels.to(model.device)
        
        # Forward pass
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss
        if batch_idx % 50 == 0:
            print(f"Batch {batch_idx} Loss: {loss.item():.4f}")

In [83]:
# Saving  model and tokenizer
model.save_pretrained('lunch_fine_tuned')
tokenizer.save_pretrained('lunch_fine_tuned')

('lunch_fine_tuned/tokenizer_config.json',
 'lunch_fine_tuned/special_tokens_map.json',
 'lunch_fine_tuned/vocab.json',
 'lunch_fine_tuned/merges.txt',
 'lunch_fine_tuned/added_tokens.json')

In [120]:
##
## TOKENIZING AND TRAINING THE WORD HOARD
##

In [116]:
with open('hoard.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    
tokenizer.pad_token = tokenizer.eos_token

hoard_text = text.replace('\n', ' ') 

hoard_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, padding=True)

print(f"Tokenized text length: {len(hoard_ids[0])}")

Tokenized text length: 1024


In [117]:
class TextDataset(Dataset):
    def __init__(self, hoard_ids, block_size):
        self.input_ids = input_ids[0]
        self.block_size = block_size
        self.num_samples = len(self.input_ids) // self.block_size  
        
        print(f"Total tokens: {len(self.input_ids)}, Number of samples: {self.num_samples}")

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        start_idx = idx * self.block_size
        end_idx = start_idx + self.block_size
        input_ids_block = self.input_ids[start_idx:end_idx]
        return input_ids_block, input_ids_block

block_size = 512 
dataset = TextDataset(input_ids, block_size)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch in dataloader:
    print(batch)

Total tokens: 1024, Number of samples: 2
[tensor([[   75,  5605,   268,  ..., 20296,  3054, 36951],
        [  610,   397,  4192,  ...,   326,  1450,   286]]), tensor([[   75,  5605,   268,  ..., 20296,  3054, 36951],
        [  610,   397,  4192,  ...,   326,  1450,   286]])]


In [126]:
# Loading gpt2 with the appropriate pad_token_id
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

# Setting model to train
model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
epochs = 32
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for batch_idx, (hoard_ids, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        # Move tensors to device (e.g., CUDA if available)
        hoard_ids = hoard_ids.to(model.device)
        labels = labels.to(model.device)
        
        # Forward pass
        outputs = model(hoard_ids, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss
        if batch_idx % 50 == 0:
            print(f"Batch {batch_idx} Loss: {loss.item():.4f}")

Epoch 1/32
Batch 0 Loss: 8.3560
Epoch 2/32
Batch 0 Loss: 8.1082
Epoch 3/32
Batch 0 Loss: 7.9214
Epoch 4/32
Batch 0 Loss: 7.7719
Epoch 5/32
Batch 0 Loss: 7.6363
Epoch 6/32
Batch 0 Loss: 7.4973
Epoch 7/32
Batch 0 Loss: 7.3744
Epoch 8/32
Batch 0 Loss: 7.2686
Epoch 9/32
Batch 0 Loss: 7.1164
Epoch 10/32
Batch 0 Loss: 6.9535
Epoch 11/32
Batch 0 Loss: 6.7981
Epoch 12/32
Batch 0 Loss: 6.6790
Epoch 13/32
Batch 0 Loss: 6.5663
Epoch 14/32
Batch 0 Loss: 6.4490
Epoch 15/32
Batch 0 Loss: 6.2862
Epoch 16/32
Batch 0 Loss: 6.1503
Epoch 17/32
Batch 0 Loss: 6.0007
Epoch 18/32
Batch 0 Loss: 5.8813
Epoch 19/32
Batch 0 Loss: 5.7458
Epoch 20/32
Batch 0 Loss: 5.6201
Epoch 21/32
Batch 0 Loss: 5.5008
Epoch 22/32
Batch 0 Loss: 5.3785
Epoch 23/32
Batch 0 Loss: 5.2259
Epoch 24/32
Batch 0 Loss: 5.1693
Epoch 25/32
Batch 0 Loss: 5.0161
Epoch 26/32
Batch 0 Loss: 4.8786
Epoch 27/32
Batch 0 Loss: 4.8445
Epoch 28/32
Batch 0 Loss: 4.6490
Epoch 29/32
Batch 0 Loss: 4.5232
Epoch 30/32
Batch 0 Loss: 4.4703
Epoch 31/32
Batch 0

In [127]:
model.save_pretrained('hoard_fine_tuned')
tokenizer.save_pretrained('hoard_fine_tuned')

('hoard_fine_tuned/tokenizer_config.json',
 'hoard_fine_tuned/special_tokens_map.json',
 'hoard_fine_tuned/vocab.json',
 'hoard_fine_tuned/merges.txt',
 'hoard_fine_tuned/added_tokens.json',
 'hoard_fine_tuned/tokenizer.json')

In [ ]:
##
## OTHER MODEL ON NAKED LUNCH 
##

In [28]:
class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size):
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()

        # Tokenizing the text
        self.examples = tokenizer(
            text, 
            return_tensors="pt", 
            max_length=block_size, 
            truncation=True
        )["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

file_path = "naked_lunch.txt"
block_size = 128

dataset = TextDataset(file_path, tokenizer, block_size)

In [45]:
from torch.utils.data import DataLoader

batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [46]:
from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

# Moving model to GPU for memory sake
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training loop
epochs = 32
model.train()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    loop = tqdm(data_loader, leave=True)
    for batch in loop:
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

/Volumes/Mac/opt/anaconda3/envs/nova/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1


Epoch 1: 100%|██████████| 1/1 [00:07<00:00,  7.12s/it, loss=2.43]


Epoch 2


Epoch 2: 100%|██████████| 1/1 [00:05<00:00,  5.86s/it, loss=2.21]


...

Epoch 30: 100%|██████████| 1/1 [00:03<00:00,  3.90s/it, loss=0.0369]


Epoch 31


Epoch 31: 100%|██████████| 1/1 [00:04<00:00,  4.90s/it, loss=0.11]


Epoch 32


Epoch 32: 100%|██████████| 1/1 [00:03<00:00,  3.96s/it, loss=0.116]


In [47]:
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")

('fine_tuned_gpt2/tokenizer_config.json',
 'fine_tuned_gpt2/special_tokens_map.json',
 'fine_tuned_gpt2/vocab.json',
 'fine_tuned_gpt2/merges.txt',
 'fine_tuned_gpt2/added_tokens.json')

In [ ]:
###
## OTHER MODEL ON WORD HOARD
##

In [141]:
class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size):
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()

        # Tokenizing the text
        self.examples = tokenizer(
            text, 
            return_tensors="pt", 
            max_length=block_size, 
            truncation=True
        )["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

file_path = "hoard.txt"
block_size = 128

dataset = TextDataset(file_path, tokenizer, block_size)

In [142]:
batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [143]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Moving model to GPU for memory sake
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training loop
epochs = 32
model.train()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    loop = tqdm(data_loader, leave=True)
    for batch in loop:
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

Epoch 1


Epoch 1: 100%|██████████| 1/1 [00:08<00:00,  8.87s/it, loss=7.69]


Epoch 2


Epoch 2: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it, loss=4.81]


Epoch 3


...

Epoch 30: 100%|██████████| 1/1 [00:03<00:00,  3.28s/it, loss=1.32]


Epoch 31


Epoch 31: 100%|██████████| 1/1 [00:03<00:00,  3.20s/it, loss=1.16]


Epoch 32


Epoch 32: 100%|██████████| 1/1 [00:03<00:00,  3.29s/it, loss=1.04]


In [146]:
model.save_pretrained("fine_tuned_hoard")
tokenizer.save_pretrained("fine_tuned_hoard")

('fine_tuned_hoard/tokenizer_config.json',
 'fine_tuned_hoard/special_tokens_map.json',
 'fine_tuned_hoard/vocab.json',
 'fine_tuned_hoard/merges.txt',
 'fine_tuned_hoard/added_tokens.json',
 'fine_tuned_hoard/tokenizer.json')

In [107]:
with open("hoard.txt", "r") as f:
    word_hoard = [line.strip() for line in f if line.strip()]

In [4]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

# Loading the fine-tuned model and the tokenizer
tokenizer = AutoTokenizer.from_pretrained("model")
model = AutoModelForCausalLM.from_pretrained("model")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generating sentences
prompt = "I walked my son to the park"
generated = generator(prompt, max_length=90, num_return_sequences=1, truncation=True)
print(f"Generated: {generated[0]['generated_text']}\n")

Device set to use cpu


Generated: I walked my son to the park when my daughter was nine. I say the words aloud: "Mom, what exactly happened?" I do not understand what happened that fateful night until this week. My daughter, who has never fully recovered, has no precise memory of the moment when she was first grabbed by the boys and thrown into the water. When I came out of the water, my daughter immediately asked what happened. She eventually took a picture of

